In [93]:
import dataclasses
import datetime
import itertools
import textwrap

In [94]:
@dataclasses.dataclass(frozen=True)
class PodcastBacklog:
    record_date: datetime.date
    num_episodes_left: int
    hours_remaining: int
    earliest_episode: datetime.date
    storage_gb: float
    num_episodes_added: int
    hours_added: int

In [95]:
RECORD_2026_01 = PodcastBacklog(
    record_date=datetime.date(year=2026, month=1, day=1),
    num_episodes_left=200,
    hours_remaining=111,
    earliest_episode=datetime.date(year=2025, month=8, day=8),
    storage_gb=7.9,
    num_episodes_added=0,
    hours_added = 0
)

RECORD_2026_02 = PodcastBacklog(
    record_date=datetime.date(year=2026, month=2, day=1),
    num_episodes_left=173,
    hours_remaining=95,
    earliest_episode=datetime.date(year=2025, month=10, day=6),
    storage_gb=6.71,
    num_episodes_added=42,
    hours_added = 24
)

In [96]:
RECORDS = (
    RECORD_2026_01,
    RECORD_2026_02,
)

assert(all(r1.record_date < r2.record_date for (r1, r2) in itertools.pairwise(RECORDS)))    

In [97]:
def bsky_message(time_left: datetime.timedelta):
    latest = RECORDS[-1]
    prev = RECORDS[-2]
    eps_played = (prev.num_episodes_left - latest.num_episodes_left) + latest.num_episodes_added
    hrs_played = (prev.hours_remaining - latest.hours_remaining) + latest.hours_added
    end_date = latest.record_date + time_left
    print(textwrap.dedent(f"""
    PODCAST BACKLOG BURNDOWN
    {latest.record_date.strftime('%B').upper()} REPORT
    - {latest.num_episodes_left} episodes to play ({latest.num_esisodes_added} in, {eps_played} out)
    - {latest.hours_remaining} hours to play ({latest.hours_added} in, {hrs_played} out)
    - Taking up {latest.storage_gb} GB
    - Dating from {latest.earliest_episode.strftime('%b %-d, %Y')}
    - Should be all caught up by {end_date.strftime('%b %-d, %Y')}"""))    

Scrap this later once there's more than two data points

In [98]:
R1 = RECORD_2026_01
R2 = RECORD_2026_02
DELTA_DATE = (R2.record_date - R1.record_date)

In [99]:
def zero_date(val1: float, val2: float) -> datetime.timedelta:
    # Units per day:
    slope = (val2 - val1)/DELTA_DATE.days
    assert(slope < 0), f'{val1}, {val2}'
    # Units over units per day --> days:
    days_left = val2/abs(slope)
    return datetime.timedelta(days=days_left)

In [100]:
delta_num_eps = zero_date(val1=float(R1.num_episodes_left), val2=float(R2.num_episodes_left))
delta_hours = zero_date(val1=float(R1.hours_remaining), val2=float(R2.hours_remaining))
delta_earliest = zero_date(
    val1=float((R1.record_date - R1.earliest_episode).days),
    val2=float((R2.record_date - R2.earliest_episode).days),
)
delta_storage = zero_date(val1=R1.storage_gb, val2=R2.storage_gb)

In [101]:
time_left = datetime.timedelta(days=sum(d.days for d in [delta_num_eps, delta_hours, delta_earliest, delta_storage]) / 4)
bsky_message(time_left)


PODCAST BACKLOG BURNDOWN
FEBRUARY REPORT
- 173 episodes to play
- 95 hours (at 1x playback)
- Taking up 6.71 GB
- Dating from Oct 6, 2025
- Should be all caught up by Jul 22, 2026
